# 03 — Demo: Transformer Tutor de Matemáticas y Física

Notebook de demostración del modelo Transformer con interfaz en **Gradio Blocks**.

1. Carga el modelo Transformer entrenado desde checkpoints
2. Generación autoregresiva con métricas **reales basadas en logits** (confianza, perplexity)
3. Interfaz interactiva con selector de dominio (math / physics)
4. Curvas de entrenamiento, resultados de evaluación y análisis de limitaciones
5. Discusión técnica honesta: qué puede y qué NO puede el modelo

In [13]:
# === Setup: GPU workaround para Blackwell + imports ===
import os, sys, subprocess

# XLA/GPU config ANTES de importar TF
os.environ.setdefault("XLA_FLAGS", "--xla_gpu_cuda_data_dir=/usr/local/cuda-12.8")
os.environ.setdefault("TF_XLA_FLAGS", "--tf_xla_auto_jit=2")

try:
    import gradio as gr
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "gradio==4.44.0", "-q"])
    import gradio as gr

import tensorflow as tf
import numpy as np
import json
import time
import re

# GPU memory growth + Blackwell cast patch
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    _original_cast = tf.cast
    def _blackwell_cast(x, dtype, name=None):
        if tf.executing_eagerly():
            with tf.device('/CPU:0'):
                return _original_cast(x, dtype, name=name)
        return _original_cast(x, dtype, name=name)
    tf.cast = _blackwell_cast

# Ruta del proyecto
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
for p in [project_root, os.path.dirname(project_root)]:
    if p not in sys.path:
        sys.path.insert(0, p)

tf.get_logger().setLevel('ERROR')

print(f"✅ TensorFlow {tf.__version__}")
print(f"✅ Gradio {gr.__version__}")
print(f"✅ GPU: {tf.config.list_physical_devices('GPU')}")
print(f"✅ Proyecto: {project_root}")

✅ TensorFlow 2.20.0
✅ Gradio 4.44.0
✅ GPU: []
✅ Proyecto: /home/melissa/transformer_math_physics_tutor


## 1. Cargar Modelo Pre-entrenado y Resultados de Evaluación

In [14]:
from transformer_math_physics_tutor.data.tokenizer import CharTokenizer
from transformer_math_physics_tutor.models.config import TransformerConfig
from transformer_math_physics_tutor.models.transformer import Transformer

# --- Cargar configuración ---
checkpoint_dir = os.path.join(project_root, 'checkpoints')
config = TransformerConfig.load(os.path.join(checkpoint_dir, 'config.json'))

# --- Cargar tokenizer ---
tokenizer = CharTokenizer(os.path.join(checkpoint_dir, 'vocab.json'))
config.vocab_size = tokenizer.vocab_size

# --- Crear e inicializar modelo ---
with tf.device('/CPU:0'):
    model = Transformer(config)
    dummy_enc = tf.zeros((1, config.max_encoder_len), dtype=tf.int32)
    dummy_dec = tf.zeros((1, config.max_decoder_len), dtype=tf.int32)
    _ = model((dummy_enc, dummy_dec), training=False)

    # Preferir best_model si existe
    best_path = os.path.join(checkpoint_dir, 'best_model.weights.h5')
    fallback_path = os.path.join(checkpoint_dir, 'model_weights.weights.h5')
    weights_path = best_path if os.path.exists(best_path) else fallback_path
    model.load_weights(weights_path)

num_params = model.count_params()
device = "GPU" if gpus else "CPU"

# --- Historial de entrenamiento ---
history = None
final_train_acc = final_val_acc = 0.0
try:
    with open(os.path.join(checkpoint_dir, 'training_history.json'), 'r') as f:
        history = json.load(f)
    final_train_acc = history['train_accuracy'][-1]
    if history.get('val_accuracy'):
        final_val_acc = history['val_accuracy'][-1]
except Exception:
    pass

# --- Reporte de evaluación ---
eval_report = None
try:
    with open(os.path.join(checkpoint_dir, 'evaluation_report.json'), 'r') as f:
        eval_report = json.load(f)
except Exception:
    pass

# --- Config dict para display ---
with open(os.path.join(checkpoint_dir, 'config.json'), 'r') as f:
    config_dict = json.load(f)

print(f"✅ Modelo cargado: {num_params:,} parámetros ({os.path.basename(weights_path)})")
print(f"✅ Dispositivo: {device}")
print(f"✅ Vocab size: {tokenizer.vocab_size} (character-level)")
print(f"✅ Accuracy — Train: {final_train_acc:.1%}, Val: {final_val_acc:.1%}")
if eval_report:
    em = eval_report.get('exact_match', {})
    print(f"✅ Exact Match (test): {em.get('correct', 0)}/{em.get('total', 0)} = {em.get('pct', 0):.1f}%")

Configuración cargada desde /home/melissa/transformer_math_physics_tutor/checkpoints/config.json
Vocabulario cargado desde /home/melissa/transformer_math_physics_tutor/checkpoints/vocab.json (135 tokens)
✅ Modelo cargado: 7,476,615 parámetros (best_model.weights.h5)
✅ Dispositivo: CPU
✅ Vocab size: 135 (character-level)
✅ Accuracy — Train: 83.3%, Val: 82.1%
✅ Exact Match (test): 0/100 = 0.0%


## 2. Función de Generación con Métricas Reales

La función `generate_with_metrics` usa decodificación **autoregresiva con top-k sampling** y calcula métricas basadas en **logits reales**:

- **Confianza**: $\text{confidence} = \frac{1}{T}\sum_{t=1}^{T} \max_v P(v \mid v_{<t})$ — promedio de la probabilidad máxima por paso
- **Perplexity**: $\text{PPL} = \exp\!\left(-\frac{1}{T}\sum_{t=1}^{T} \log P(v_t \mid v_{<t})\right)$ — sobre la secuencia seleccionada
- **Repetition penalty**: Penaliza tokens ya generados para evitar bucles

In [15]:
def _pad_to_length(tokens, max_len, pad_id):
    """Paddea o trunca una secuencia a max_len."""
    if len(tokens) > max_len:
        return tokens[:max_len - 1] + [tokens[-1]]
    return tokens + [pad_id] * (max_len - len(tokens))


def _detect_ngram_repeat(tokens, n=8):
    """Detecta si los últimos n tokens se repiten antes."""
    if len(tokens) < n * 2:
        return False
    last_ngram = tokens[-n:]
    for i in range(len(tokens) - n * 2, max(0, len(tokens) - n * 5) - 1, -1):
        if tokens[i:i + n] == last_ngram:
            return True
    return False


def generate_with_metrics(problem, max_length=300, temperature=0.3,
                          top_k=10, repetition_penalty=1.3):
    """
    Genera respuesta con métricas reales basadas en logits.

    Usa la misma lógica de generación que inference/generate.py
    (padding, top-k, repetition penalty) + cálculo de métricas.

    Returns dict con: text, confidence, perplexity, inference_time, num_tokens
    """
    if not problem or not problem.strip():
        return {
            'text': "⚠️ Escribe un problema de matemáticas o física.",
            'confidence': 0.0, 'perplexity': 0.0,
            'inference_time': 0.0, 'num_tokens': 0
        }

    start_time = time.time()

    # Tokenizar y paddear encoder input (misma longitud que entrenamiento)
    encoder_tokens = tokenizer.encode(problem, add_special_tokens=True)
    max_enc_len = config.max_encoder_len  # 200
    encoder_tokens = _pad_to_length(encoder_tokens, max_enc_len, tokenizer.pad_token_id)
    encoder_input = tf.constant([encoder_tokens], dtype=tf.int32)  # (1, 200)

    # Inicializar decoder
    decoder_tokens = [tokenizer.start_token_id]
    generated_tokens = []
    token_probs = []

    for _ in range(max_length):
        decoder_input = tf.constant([decoder_tokens], dtype=tf.int32)

        # Forward pass -> logits
        predictions = model((encoder_input, decoder_input), training=False)
        logits = predictions[0, -1, :]  # (vocab_size,)

        # Probabilidades ANTES de manipulación (para métricas puras)
        probs = tf.nn.softmax(logits).numpy()
        max_prob = float(np.max(probs))

        # Repetition penalty
        logits_np = logits.numpy()
        if repetition_penalty > 1.0 and generated_tokens:
            for tok_id in set(generated_tokens):
                if logits_np[tok_id] > 0:
                    logits_np[tok_id] /= repetition_penalty
                else:
                    logits_np[tok_id] *= repetition_penalty

        # Selección de token con top-k sampling
        if temperature <= 0.0:
            predicted_id = int(np.argmax(logits_np))
        else:
            scaled = logits_np / temperature
            if top_k > 0:
                top_indices = np.argpartition(scaled, -top_k)[-top_k:]
                mask = np.full_like(scaled, -1e9)
                mask[top_indices] = scaled[top_indices]
                scaled = mask
            exp_scaled = np.exp(scaled - np.max(scaled))
            sampling_probs = exp_scaled / exp_scaled.sum()
            predicted_id = int(np.random.choice(len(sampling_probs), p=sampling_probs))

        # Probabilidad del token seleccionado (de distribución original)
        selected_prob = float(probs[predicted_id])
        token_probs.append(selected_prob)

        if predicted_id == tokenizer.end_token_id:
            break

        generated_tokens.append(predicted_id)

        if len(generated_tokens) > 20 and _detect_ngram_repeat(generated_tokens, n=10):
            break

        decoder_tokens.append(predicted_id)

    inference_time = time.time() - start_time

    # Decodificar
    generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()

    # Métricas
    if token_probs:
        confidence = float(np.mean(token_probs))
        nll = -np.mean([np.log(p + 1e-10) for p in token_probs])
        perplexity = float(np.exp(min(nll, 20)))  # Cap para evitar overflow
    else:
        confidence = 0.0
        perplexity = 0.0

    return {
        'text': generated_text,
        'confidence': confidence,
        'perplexity': perplexity,
        'inference_time': inference_time,
        'num_tokens': len(generated_tokens)
    }


# === Test rápido (math + physics) ===
print("=" * 60)
print("TEST RÁPIDO — Math")
print("=" * 60)
r = generate_with_metrics("Janet has 3 apples and buys 5 more. How many does she have?")
print(f"  Respuesta: {r['text'][:200]}")
print(f"  Confianza: {r['confidence']:.2%} | PPL: {r['perplexity']:.2f} | {r['num_tokens']} tokens | {r['inference_time']*1000:.0f}ms")

print("\nTEST RÁPIDO — Physics")
r2 = generate_with_metrics("A car accelerates from rest at 3 m/s² for 5 seconds. What is its final velocity?")
print(f"  Respuesta: {r2['text'][:200]}")
print(f"  Confianza: {r2['confidence']:.2%} | PPL: {r2['perplexity']:.2f} | {r2['num_tokens']} tokens | {r2['inference_time']*1000:.0f}ms")

TEST RÁPIDO — Math
  Respuesta: Step 1: There are 40 - 24 = <<40-24=36>>36 slices left.
Step 2: There are 50 - 36 = <<50-36=14>>14 slices left.
Answer: 14
  Confianza: 83.30% | PPL: 1.36 | 122 tokens | 18208ms

TEST RÁPIDO — Physics
  Respuesta: Step 1: There are 250 minutes in all when 350 minutes
  Confianza: 73.76% | PPL: 1.67 | 53 tokens | 6821ms


In [16]:
# === Diagnóstico: probar con problemas de ambos dominios ===
test_problems = [
    ("Janet has 3 apples and buys 5 more. How many apples does she have?", "math"),
    ("A store sells pencils for $2 each. Tom buys 7 pencils. How much does he spend?", "math"),
    ("A train travels 60 miles per hour for 3 hours. How far does it go?", "math"),
    ("Sarah has 24 cookies and wants to share them equally among 6 friends. How many cookies does each friend get?", "math"),
    ("John has $50. He buys a book for $12 and a pen for $3. How much money does he have left?", "math"),
    ("A car accelerates from rest at 3 m/s² for 5 seconds. What is its final velocity?", "physics"),
    ("A 10 kg box is pushed with a force of 50 N. What is its acceleration?", "physics"),
    ("How much heat is needed to raise the temperature of 2 kg of water by 30°C?", "physics"),
    ("A circuit has a voltage of 12V and a resistance of 4Ω. What is the current?", "physics"),
    ("An object is dropped from 80 m. How long does it take to hit the ground?", "physics"),
]

print("=" * 70)
print("DIAGNÓSTICO DEL MODELO — Math + Physics")
print("=" * 70)
for problem, domain in test_problems:
    r = generate_with_metrics(problem)
    if r['confidence'] >= 0.8:
        status = "🟢"
    elif r['confidence'] >= 0.5:
        status = "🟡"
    else:
        status = "🔴"
    print(f"\n{status} [{domain.upper()}] {problem}")
    print(f"   Respuesta: {r['text'][:200]}")
    print(f"   Confianza: {r['confidence']:.2%} | PPL: {r['perplexity']:.2f} | {r['num_tokens']} tokens")

DIAGNÓSTICO DEL MODELO — Math + Physics

🟢 [MATH] Janet has 3 apples and buys 5 more. How many apples does she have?
   Respuesta: Step 1: There are 30 + 4 = <<30+4=72>>72 students in the former that were children.
Step 2: There are 72 / 5 = <<72/5=14>>14 students in the second that were children.
Answer: 14
   Confianza: 81.55% | PPL: 1.37 | 178 tokens

🟢 [MATH] A store sells pencils for $2 each. Tom buys 7 pencils. How much does he spend?
   Respuesta: Step 1: The first combined amount of money that is $2.00 x 2 = $<<2.00*2=4.00>>4.00.
Step 2: The second combined amount of money that is $4.00 x 2 = $<<4.00*2=8.00>>8.00.
Step 3: The second combined a
   Confianza: 87.44% | PPL: 1.20 | 300 tokens

🟢 [MATH] A train travels 60 miles per hour for 3 hours. How far does it go?
   Respuesta: Step 1: There are 24 students in the first class, so there are 24/2=<<24/2=12>>12 students in the second class.
Step 2: There are 24 students in the first class, so there are 24/12=<<24/12=2>>2 studen
  

## 2.5 Diagnóstico Profundo — ¿Qué aprendió el modelo?

Vamos a investigar tres hipótesis:
1. **¿El encoder influye en la salida?** — Si cambiamos el problema, ¿cambia la respuesta?
2. **¿Greedy vs sampling?** — ¿El ruido de sampling empeora las cosas?
3. **¿Teacher forcing ayuda?** — Si le damos los primeros tokens correctos, ¿puede continuar?
4. **¿Cross-attention funciona?** — ¿El decoder atiende al encoder?

In [18]:
print("=" * 70)
print("DIAGNÓSTICO 1: ¿El encoder influye en la salida?")
print("  (Mismo decoder start, dos problemas distintos, greedy)")
print("=" * 70)

for prob_text in [
    "What is 2 + 3?",
    "A farmer has 100 cows and sells 30. How many cows remain?",
    "A ball is thrown upward with a velocity of 20 m/s. What is the maximum height?",
]:
    r = generate_with_metrics(prob_text, temperature=0.0, top_k=0)
    print(f"\n  Problema: {prob_text}")
    print(f"  Respuesta (greedy): {r['text'][:250]}")
    print(f"  Confianza: {r['confidence']:.2%} | Tokens: {r['num_tokens']}")

print("\n" + "=" * 70)
print("DIAGNÓSTICO 2: ¿El encoder produce embeddings distintos?")
print("=" * 70)

problems_to_compare = [
    "What is 5 + 3?",
    "A car moves at 10 m/s for 5 seconds. What distance does it travel?",
    "XYZXYZXYZ",
]

enc_outputs = []
for prob in problems_to_compare:
    enc_tokens = tokenizer.encode(prob, add_special_tokens=True)
    enc_padded = _pad_to_length(enc_tokens, config.max_encoder_len, tokenizer.pad_token_id)
    enc_input = tf.constant([enc_padded], dtype=tf.int32)
    enc_padding_mask = model.create_padding_mask(enc_input)
    enc_out = model.encoder(enc_input, training=False, mask=enc_padding_mask)
    enc_outputs.append(enc_out.numpy())
    print(f"  '{prob[:50]}' → mean={enc_out.numpy().mean():.4f}, "
          f"std={enc_out.numpy().std():.4f}, norm={np.linalg.norm(enc_out.numpy()):.2f}")

from numpy.linalg import norm
for i in range(len(enc_outputs)):
    for j in range(i+1, len(enc_outputs)):
        a, b = enc_outputs[i].flatten(), enc_outputs[j].flatten()
        cos_sim = np.dot(a, b) / (norm(a) * norm(b) + 1e-8)
        print(f"  Cosine sim [{i}] vs [{j}]: {cos_sim:.4f}")

print("\n" + "=" * 70)
print("DIAGNÓSTICO 3: ¿Cross-attention pesos?")
print("=" * 70)

prob = "What is 5 + 3?"
enc_tokens = tokenizer.encode(prob, add_special_tokens=True)
enc_padded = _pad_to_length(enc_tokens, config.max_encoder_len, tokenizer.pad_token_id)
enc_input = tf.constant([enc_padded], dtype=tf.int32)
dec_input = tf.constant([[tokenizer.start_token_id]], dtype=tf.int32)

enc_padding_mask = model.create_padding_mask(enc_input)
dec_padding_mask = model.create_padding_mask(enc_input)
look_ahead_mask = model.create_look_ahead_mask(tf.shape(dec_input)[1])
dec_target_padding_mask = model.create_padding_mask(dec_input)
combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

enc_output = model.encoder(enc_input, training=False, mask=enc_padding_mask)
dec_output, attn_weights = model.decoder(
    dec_input, enc_output, training=False,
    look_ahead_mask=combined_mask, padding_mask=dec_padding_mask
)

n_real = len(enc_tokens)
for layer_name, weights in attn_weights.items():
    if 'block2' in layer_name:
        w = weights.numpy()[0]  # (heads, tar_len, inp_len)
        real_w = w[:, :, :n_real]
        pad_w = w[:, :, n_real:]
        total = real_w.sum() + pad_w.sum()
        print(f"  {layer_name}: tokens reales={real_w.sum()/total:.1%}, padding={pad_w.sum()/total:.1%}")
        avg = w.mean(axis=0)[0, :n_real]
        top_pos = avg.argsort()[-5:][::-1]
        chars = [tokenizer.decode([enc_padded[p]], skip_special_tokens=False) for p in top_pos]
        entropy = -np.sum(avg * np.log(avg + 1e-10))
        max_entropy = np.log(n_real)
        print(f"    Entropy: {entropy:.3f} / {max_entropy:.3f} (max) = {entropy/max_entropy:.1%} → "
              f"{'⚠️ UNIFORME' if entropy/max_entropy > 0.95 else '✅ Selectiva'}")
        print(f"    Top-5: {list(zip(chars, [f'{avg[p]:.4f}' for p in top_pos]))}")

print("\n" + "=" * 70)
print("DIAGNÓSTICO 4: Teacher forcing — ¿puede continuar?")
print("=" * 70)

prob = "What is 5 + 3?"
partial = "Step 1: 5 + 3 = "
enc_tokens = tokenizer.encode(prob, add_special_tokens=True)
enc_padded = _pad_to_length(enc_tokens, config.max_encoder_len, tokenizer.pad_token_id)
enc_input = tf.constant([enc_padded], dtype=tf.int32)

sol_tokens = [tokenizer.start_token_id] + tokenizer.encode(partial, add_special_tokens=False)
generated = list(sol_tokens)
max_dec_len = config.max_decoder_len  # 300

for _ in range(100):
    if len(generated) >= max_dec_len:
        break
    dec_input = tf.constant([generated], dtype=tf.int32)
    predictions = model((enc_input, dec_input), training=False)
    next_id = int(np.argmax(predictions[0, -1, :].numpy()))
    if next_id == tokenizer.end_token_id:
        break
    generated.append(next_id)

full_text = tokenizer.decode(generated[1:], skip_special_tokens=True)
print(f"  Problema: '{prob}'")
print(f"  Teacher forcing: '{partial}'")
print(f"  Continuación: '{full_text[:200]}'")

print("\n" + "=" * 70)
print("═══ RESUMEN DEL DIAGNÓSTICO ═══")
print("=" * 70)
print("""
  HALLAZGO PRINCIPAL: Cross-attention COLAPSADA a distribución uniforme.

  ¿Qué significa?
  → El decoder NO lee el encoder. Genera texto "genérico" como un
    language model, sin condicionarse en el problema de entrada.
  → Todas las capas tienen entropy ≈ 100% (distribución uniforme).

  ¿Por qué ocurre?
  → Con tokenización char-level, el decoder puede predecir el
    siguiente carácter con alta accuracy (~82%) usando SOLO el
    contexto autoregresivo previo, sin necesitar cross-attention.
  → Ejemplo: después de "Ste", el modelo predice "p" sin mirar
    el encoder. Después de "Step ", predice "1" o "2", etc.

  SOLUCIÓN: Aplicar "decoder token masking" durante entrenamiento.
  → Reemplazar aleatoriamente 15-20% de los tokens del decoder input
    con un token <MASK> durante training.
  → Esto ROMPE el atajo autoregresivo y FUERZA al modelo a usar
    cross-attention para obtener la información faltante.
""")
print("=" * 70)

DIAGNÓSTICO 1: ¿El encoder influye en la salida?
  (Mismo decoder start, dos problemas distintos, greedy)

  Problema: What is 2 + 3?
  Respuesta (greedy): Step 1: The first term is a congruent that the second term is a
  Confianza: 77.61% | Tokens: 63

  Problema: A farmer has 100 cows and sells 30. How many cows remain?
  Respuesta (greedy): Step 1: There are 24 considers in the first two terms, so there are 24 considers in the second term.
Step 2: There are 24 considers in the second term.
Answer: 24
  Confianza: 81.03% | Tokens: 162

  Problema: A ball is thrown upward with a velocity of 20 m/s. What is the maximum height?
  Respuesta (greedy): Step 1: There are 24 ways to clean the first two times that the first
  Confianza: 75.85% | Tokens: 69

DIAGNÓSTICO 2: ¿El encoder produce embeddings distintos?
  'What is 5 + 3?' → mean=0.0031, std=0.0403, norm=9.15
  'A car moves at 10 m/s for 5 seconds. What distance' → mean=0.0016, std=0.0446, norm=10.10
  'XYZXYZXYZ' → mean=0.0020, std=

## 3. Interfaz Gradio Blocks — Demo para el Profesor

Layout:
- **Columna izquierda**: Entrada del problema, selector de dominio, botón de resolver, solución generada, respuesta destacada
- **Columna derecha**: Métricas de generación, detalles de arquitectura
- **Acordeones**: Detalles técnicos, curvas de entrenamiento, evaluación, limitaciones

In [ ]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import tempfile


def create_training_plot():
    """Crea gráfica de entrenamiento y la guarda como archivo temporal."""
    if not history:
        return None
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 3))
    ax1.plot(history['train_loss'], label='Train', linewidth=2, color='#4f46e5')
    if history.get('val_loss'):
        ax1.plot(history['val_loss'], label='Val', linewidth=2, color='#f97316')
    ax1.set_title('Loss', fontsize=13, fontweight='bold')
    ax1.set_xlabel('Época')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax2.plot(history['train_accuracy'], label='Train', linewidth=2, color='#4f46e5')
    if history.get('val_accuracy'):
        ax2.plot(history['val_accuracy'], label='Val', linewidth=2, color='#f97316')
    ax2.set_title('Accuracy (token-level)', fontsize=13, fontweight='bold')
    ax2.set_xlabel('Época')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    plt.tight_layout()
    tmp = tempfile.NamedTemporaryFile(suffix='.png', delete=False)
    plt.savefig(tmp.name, format='png', dpi=100, bbox_inches='tight')
    plt.close(fig)
    return tmp.name


def extract_answer_highlight(text):
    """Extrae la línea Answer: y la resalta."""
    match = re.search(r"(Answer:\s*.+?)(?:\n|$)", text, re.IGNORECASE)
    if match:
        return match.group(1).strip()
    return "No se detectó línea Answer:"


def tutor_interface(problem, domain):
    """Wrapper para Gradio: genera respuesta con métricas."""
    # Agregar contexto de dominio si el usuario lo selecciona
    if domain == "physics" and "physics" not in problem.lower():
        prompt = problem  # El modelo debe inferir el dominio
    else:
        prompt = problem

    result = generate_with_metrics(prompt, max_length=300)

    # Indicador visual
    if result['confidence'] >= 0.8:
        conf_indicator = "🟢 Alta"
    elif result['confidence'] >= 0.5:
        conf_indicator = "🟡 Media"
    else:
        conf_indicator = "🔴 Baja"

    # Extraer Answer:
    answer_line = extract_answer_highlight(result['text'])

    return (
        result['text'],            # solution
        f"**{answer_line}**",      # answer highlight
        result['confidence'],      # slider
        conf_indicator,            # status
        f"{result['perplexity']:.2f}",
        f"{result['inference_time']*1000:.0f}",
        result['num_tokens']
    )


# === Dataset stats ===
dataset_stats = ""
try:
    with open(os.path.join(project_root, 'data', 'combined_math_physics.json'), 'r') as f:
        all_data = json.load(f)
    n_math = sum(1 for d in all_data if d.get('domain') == 'math')
    n_phys = sum(1 for d in all_data if d.get('domain') == 'physics')
    n_train = sum(1 for d in all_data if d.get('split') == 'train')
    dataset_stats = f"Dataset: **{len(all_data):,}** problemas ({n_math:,} math + {n_phys:,} physics) | Train: **{n_train:,}**"
except Exception:
    dataset_stats = "Dataset: combined_math_physics.json"

# === Evaluation summary for header ===
eval_summary = ""
if eval_report:
    ta = eval_report.get('token_accuracy', {})
    em = eval_report.get('exact_match', {})
    eval_summary = f"Token Acc (test): **{ta.get('test_acc', 0):.1%}** | Exact Match: **{em.get('pct', 0):.1f}%**"


# ═══════════════════════════════════════════════════════════
#  INTERFAZ GRADIO BLOCKS
# ═══════════════════════════════════════════════════════════
theme = gr.themes.Soft(primary_hue="indigo", secondary_hue="blue")

with gr.Blocks(theme=theme, title="Transformer Tutor — Math & Physics") as demo:

    # ── Header ─────────────────────────────────────────────
    gr.Markdown(f"""
    # 🎓 Transformer Tutor — Matemáticas y Física

    Modelo **Encoder-Decoder Transformer** desde cero |
    **{num_params:,}** parámetros | Dispositivo: **{device}** |
    Train: **{final_train_acc:.1%}** | Val: **{final_val_acc:.1%}** |
    {eval_summary}

    {dataset_stats}
    """)

    with gr.Row():

        # ═══ COLUMNA IZQUIERDA: Entrada / Salida ═══
        with gr.Column(scale=3):

            with gr.Row():
                input_box = gr.Textbox(
                    label="📝 Problema",
                    placeholder="Ej: A car accelerates from rest at 3 m/s² for 5 seconds. What is its final velocity?",
                    lines=3,
                    scale=4
                )
                domain_selector = gr.Radio(
                    choices=["math", "physics"],
                    value="math",
                    label="🔬 Dominio",
                    scale=1
                )

            with gr.Row():
                solve_btn = gr.Button("🤔 Resolver", variant="primary", scale=3)
                clear_btn = gr.Button("🗑️ Limpiar", variant="secondary", scale=1)

            output_box = gr.Textbox(
                label="🤖 Solución (paso a paso)",
                lines=8,
                show_copy_button=True
            )

            answer_box = gr.Markdown(
                label="Respuesta",
                value="*Esperando problema...*"
            )

            # Ejemplos por dominio
            gr.Markdown("### 📚 Ejemplos")
            with gr.Tab("Matemáticas"):
                gr.Examples(
                    examples=[
                        ["Janet has 3 apples and buys 5 more. How many apples does she have?", "math"],
                        ["A store sells pencils for $2 each. Tom buys 7 pencils. How much does he spend?", "math"],
                        ["Sarah has 24 cookies and shares equally among 6 friends. How many does each get?", "math"],
                        ["A farmer has 15 chickens. Each lays 2 eggs per day. How many eggs in 3 days?", "math"],
                        ["John has $50. He buys a book for $12 and a pen for $3. How much is left?", "math"],
                    ],
                    inputs=[input_box, domain_selector],
                    label=""
                )
            with gr.Tab("Física"):
                gr.Examples(
                    examples=[
                        ["A car accelerates from rest at 3 m/s² for 5 seconds. What is its final velocity?", "physics"],
                        ["A 10 kg box is pushed with a force of 50 N. What is its acceleration?", "physics"],
                        ["How much heat is needed to raise the temperature of 2 kg of water by 30°C?", "physics"],
                        ["A circuit has voltage 12V and resistance 4Ω. What is the current?", "physics"],
                        ["An object is dropped from 80 m. How long does it take to hit the ground?", "physics"],
                    ],
                    inputs=[input_box, domain_selector],
                    label=""
                )

        # ═══ COLUMNA DERECHA: Métricas ═══
        with gr.Column(scale=2):

            gr.Markdown("### 📊 Métricas de Generación")
            with gr.Group():
                conf_slider = gr.Slider(0, 1, value=0, label="Confianza", interactive=False)
                conf_text = gr.Textbox(label="Estado", value="⏳ Esperando...", interactive=False)
                perp_text = gr.Textbox(label="Perplexity", value="--", interactive=False)
                time_text = gr.Textbox(label="Tiempo (ms)", value="--", interactive=False)
                tokens_num = gr.Number(label="Tokens generados", value=0)

            gr.Markdown("### 🧠 Arquitectura")
            with gr.Row():
                gr.Textbox(str(config_dict['num_layers']), label="Capas", interactive=False)
                gr.Textbox(str(config_dict['d_model']), label="d_model", interactive=False)
            with gr.Row():
                gr.Textbox(str(config_dict['num_heads']), label="Heads", interactive=False)
                gr.Textbox(str(config_dict.get('dff', 1024)), label="dff", interactive=False)
            with gr.Row():
                gr.Textbox(str(config_dict['dropout_rate']), label="Dropout", interactive=False)
                gr.Textbox(str(config_dict['vocab_size']), label="Vocab (chars)", interactive=False)

    # ── Acordeón: Detalles Técnicos ───────────────────────
    with gr.Accordion("🔍 Detalles Técnicos", open=False):
        gr.Markdown(f"""
**Componentes implementados desde cero (sin usar capas pre-hechas):**
- Scaled Dot-Product Attention: `softmax(QK^T / √d_k) V`
- Multi-Head Attention ({config_dict['num_heads']} cabezas, depth={config_dict['d_model'] // config_dict['num_heads']})
- Positional Encoding sinusoidal
- Encoder: Self-Attn → Add&Norm → FFN → Add&Norm (×{config_dict['num_layers']})
- Decoder: Masked Self-Attn → Cross-Attn → FFN (×{config_dict['num_layers']})

**Dataset combinado:**
- GSM8K: ~8,600 problemas de aritmética con soluciones paso a paso
- MATH (LLM-solved): ~1,900 problemas de álgebra, combinatoria, geometría
- Física (templates): ~2,000 problemas de cinemática, dinámica, termodinámica, etc.
- Total: ~12,500 problemas después de filtrado de calidad

**Tokenización:** Character-level ({config_dict['vocab_size']} tokens: 131 ASCII + 4 especiales)

**Entrenamiento:**
- Optimizador: Adam (β₁=0.9, β₂=0.98) con LR warmup ({config_dict['warmup_steps']} pasos)
- Loss: SparseCategoricalCrossentropy con label smoothing ({config_dict.get('label_smoothing', 0.1)})
- Épocas: {len(history['train_loss']) if history else '?'} (early stopping patience=10)
- GPU: NVIDIA RTX 5060 (Blackwell) con workarounds XLA

**Inferencia:**
- Decodificación: Top-k sampling (k=10, temperature=0.3)
- Repetition penalty: 1.3 (penaliza tokens ya generados)
- Detección de n-gram repeat para evitar bucles
        """)

    # ── Acordeón: Curvas de Entrenamiento ─────────────────
    if history:
        with gr.Accordion("📈 Curvas de Entrenamiento", open=False):
            plot_img = create_training_plot()
            if plot_img:
                gr.Image(value=plot_img, label="Historia de entrenamiento")
            gr.Markdown(f"""
**Resumen del entrenamiento:**
- Épocas completadas: {len(history['train_loss'])}
- Loss final — Train: {history['train_loss'][-1]:.4f} | Val: {history.get('val_loss', [0])[-1]:.4f}
- Accuracy final — Train: {history['train_accuracy'][-1]:.4f} | Val: {history.get('val_accuracy', [0])[-1]:.4f}
- Mejor val_accuracy en época: {np.argmax(history.get('val_accuracy', [0])) + 1 if history.get('val_accuracy') else 'N/A'}
            """)

    # ── Acordeón: Evaluación ──────────────────────────────
    with gr.Accordion("📋 Resultados de Evaluación", open=False):
        if eval_report:
            ta = eval_report.get('token_accuracy', {})
            em = eval_report.get('exact_match', {})
            eval_md = f"""
**Token-level Accuracy** (predicción del siguiente carácter):
- Validación: {ta.get('val_acc', 0):.1%}
- Test: {ta.get('test_acc', 0):.1%}

**Exact Match (línea Answer:)** sobre {em.get('total', 0)} problemas de test:
- Global: **{em.get('correct', 0)}/{em.get('total', 0)} = {em.get('pct', 0):.1f}%**
"""
            for domain, stats in sorted(em.get('by_domain', {}).items()):
                eval_md += f"- {domain}: {stats['correct']}/{stats['total']} = {stats['correct']/max(stats['total'],1)*100:.1f}%\n"

            eval_md += "\n**Ejemplos de predicciones:**\n"
            for ex in eval_report.get('examples', [])[:6]:
                icon = "✅" if ex['correct'] else "❌"
                eval_md += f"- {icon} [{ex['domain']}] \"{ex['problem'][:80]}...\" → Esperado: `{ex['ref_answer']}` | Predicho: `{ex['pred_answer']}`\n"

            gr.Markdown(eval_md)
        else:
            gr.Markdown("*No se encontró `evaluation_report.json`. Ejecuta `evaluation/evaluate_math_physics.py`.*")

    # ── Acordeón: Limitaciones ────────────────────────────
    with gr.Accordion("⚠️ Limitaciones y Análisis Honesto", open=True):
        gr.Markdown("""
### ¿Qué puede y qué NO puede este modelo?

**✅ Lo que el modelo SÍ demuestra:**
1. **Arquitectura Transformer correcta**: Encoder-Decoder completo implementado desde cero
2. **Aprendizaje de formato**: El modelo genera consistentemente el patrón "Step 1: ... Step 2: ... Answer: ..."
3. **Pipeline completo de ML**: Datos → tokenización → entrenamiento → evaluación → demo
4. **Alta token accuracy (82%)**: Predice bien el siguiente carácter dado el contexto

**❌ Lo que el modelo NO puede hacer:**
1. **Razonamiento matemático**: Las respuestas numéricas son incorrectas (0% exact match)
2. **Alineación problema→solución**: El modelo genera soluciones "genéricas" sin conectar con el problema específico

### ¿Por qué ocurre esto?

El problema principal es el **tokenizer a nivel de carácter** combinado con un modelo de **7M parámetros**:

| Aspecto | Nuestro modelo | GPT-2 (referencia) |
|---------|---------------|-------------------|
| Tokenización | Character-level (135 tokens) | BPE (~50,000 tokens) |
| Parámetros | 7.4M | 117M (small) |
| Representación de "25" | 2 tokens: `'2'`, `'5'` | 1 token: `'25'` |
| Representación de "Step" | 4 tokens: `'S'`, `'t'`, `'e'`, `'p'` | 1 token: `'Step'` |

Con tokenización por carácter, un problema de 100 palabras se convierte en **~500 tokens**.
El modelo debe "recordar" relaciones entre caracteres individuales a lo largo de secuencias
de 200-300 pasos — una tarea extremadamente difícil para 7M parámetros.

### ¿Qué se necesitaría para mejorar?

1. **Tokenizer BPE/WordPiece**: Reducir secuencias ~5x, permitiendo al modelo operar a nivel de concepto
2. **Más parámetros**: 50M+ para capturar relaciones semánticas
3. **Pre-entrenamiento**: El modelo necesita comprensión lingüística previa (como BERT/GPT)
4. **Fine-tuning supervisado con chain-of-thought**: Ejemplos de razonamiento paso a paso alineados

### Valor pedagógico

A pesar de las limitaciones en razonamiento, este proyecto demuestra:
- Implementación from-scratch de un Transformer completo
- Pipeline de datos robusto (schema, validación, filtrado)
- Entrenamiento con GPU y técnicas modernas (label smoothing, LR scheduling, early stopping)
- Evaluación honesta con métricas apropiadas
- Despliegue con interfaz interactiva
        """)

    # ── Eventos ───────────────────────────────────────────
    solve_btn.click(
        fn=tutor_interface,
        inputs=[input_box, domain_selector],
        outputs=[output_box, answer_box, conf_slider, conf_text, perp_text, time_text, tokens_num]
    )

    clear_btn.click(
        fn=lambda: ("", "math", "", "*Esperando problema...*", 0, "⏳ Esperando...", "--", "--", 0),
        inputs=None,
        outputs=[input_box, domain_selector, output_box, answer_box, conf_slider, conf_text, perp_text, time_text, tokens_num]
    )

# Lanzar
demo.launch(inline=True, share=False)

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


/home/melissa/.local/lib/python3.10/site-packages/gradio/analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.44.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(
